### Yield Curve Model Optimization work

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fredapi as fa

import statsmodels.api as sm

In [2]:
# get the optimal factors on 1 of 3 criteria
def get_fact(model,returns,sort):
    df = []
    cycle_df = returns[['Mkt-RF','SMB','HML','RMW','CMA','Mom',model]].apply(pd.to_numeric, errors='coerce').dropna()
    cycle_df[model] = cycle_df[model].round()
    for i in range(1,5):
        des = cycle_df.loc[cycle_df[model]==i].describe().drop([model,'Mkt-RF'], axis=1)
        if sort == 'Sharpe':
            optimal = (des.loc['mean']/des.loc['std']).idxmax()
        elif sort == 'Return':
            optimal = des.loc['mean'].idxmax()
        elif sort == 'VaR':
            optimal = cycle_df.loc[cycle_df[model]==1].drop([model,'Mkt-RF'], axis=1).quantile(0.05).idxmax()
        df.append(optimal)
    return df  

def returns_data(model, returns, opt,optw):    
    strat = returns[['Mkt-RF','SMB','HML','RMW','CMA','Mom',model]].apply(pd.to_numeric, errors='coerce').dropna()
    strat[model] = strat[model].round()
    #set up empy cells with for the weights
    for col in strat.columns:
        if 'cycle' in col:
            continue
        if 'Mkt' in col:
            continue
        else:
            strat[col + '_weight'] = 0
    #weight the cycle returns
    for col in strat.columns:
        if ('weight' in col) & (opt[0] in col):
            strat[col] = np.where(strat[model] == 1,optw[0],strat[col])
        if ('weight' in col) & (opt[1] in col):
            strat[col] = np.where(strat[model] == 2,optw[1],strat[col])
        if ('weight' in col) & (opt[2] in col):
            strat[col] = np.where(strat[model] == 3,optw[2],strat[col])
        if ('weight' in col) & (opt[3] in col):
            strat[col] = np.where(strat[model] == 4,optw[3],strat[col])

    #combine the returns
    strat['Rotation_return'] = strat['SMB']*strat['SMB_weight'] + \
                                strat['HML']*strat['HML_weight'] + \
                                strat['RMW']*strat['RMW_weight'] + \
                                strat['CMA']*strat['CMA_weight'] + \
                                strat['Mom']*strat['Mom_weight']
    strat = strat[['Mkt-RF','Rotation_return',model]]   

    #get cumaltive returns
    strat['Market'] = (strat['Mkt-RF']+1).cumprod()-1
    strat['Strategy'] = (strat['Rotation_return']+1).cumprod()-1
    return strat

#determine best returns by taking mode of best strategies looking at 4th through 0-1/4 to 1/4-1/2
def get_opt_overtime(model,returns,sort):
    df = returns[['Mkt-RF','SMB','HML','RMW','CMA','Mom',model]].apply(pd.to_numeric, errors='coerce').dropna()
    df_4th = round(len(df)/4)
    out_return = []
    in_return = []
    for x in range(df_4th):
        in_sample = df[0+x:df_4th+x]
        out_return.append(df[df_4th+x+1:])
        in_return.append(get_fact(model,in_sample,sort))
    opt = pd.DataFrame(in_return).mode().values.tolist()[0]
    return opt, out_return[-1]

def stats(df, time,market=False):
    #in qtrs may change to monthly
    mkt_vol = df['Mkt-RF'].std() * np.sqrt(time)
    mkt_ret = (df['Market'][-1]+1)**(1/(len(df)/time))-1
    mkt_sharpe = mkt_ret/mkt_vol

    strat_vol = df['Rotation_return'].std() * np.sqrt(time)
    strat_ret = (df['Strategy'][-1]+1)**(1/(len(df)/time))-1
    strat_sharpe = strat_ret/strat_vol


    if market:
        print("Market Return: {}%".format((100*mkt_ret).round(2)))
        print("Market Vol: {}%".format((100*mkt_vol).round(2)))
        print("Market Sharpe: {}".format(mkt_sharpe.round(2)))
        print("")
    else:
        #output important results
        print("Strategy Return: {}%".format((100*strat_ret).round(2)))
        print("Strategy Vol: {}%".format((100*strat_vol).round(2)))
        print("Strategy Sharpe: {}".format(strat_sharpe.round(2)))

In [3]:
def model_describe(model,returns):
    dfs = []
    cycle_df = returns[['Mkt-RF','SMB','HML','RMW','CMA','Mom',model]].apply(pd.to_numeric, errors='coerce').dropna()
    cycle_df[model] = cycle_df[model].round()
    for x in range(1,5):
        cycle = cycle_df.loc[cycle_df[model]==x]
        count = cycle_df.loc[cycle_df[model]==x].describe()[model][0]
        #print("Descriptive Stats during cycle {} which is {}/{}".format(x, count, len(cycle_df)))
        #print(cycle.describe()[['Mkt-RF','SMB','HML','RMW','CMA','Mom']].round(4))
        dfs.append(cycle.describe()[['Mkt-RF','SMB','HML','RMW','CMA','Mom']].round(4))
    return dfs

In [4]:
facts = pd.read_csv("Factors.csv",header=3)
facts = facts.drop("String",axis=1)
facts['Date'] = pd.to_datetime(facts['Date'])
facts = facts.set_index("Date")
facts = facts.apply(pd.to_numeric)/100

#resample to qtrs to match the
facts += 1
facts = facts.rolling(3).apply(lambda x: x.prod()-1).dropna()

yield_mod =pd.read_csv('monthly.csv',usecols=['Date','Label'])
yield_mod['Date'] = pd.to_datetime(yield_mod['Date'])
yield_mod['Label'] = yield_mod['Label'] + 1
yield_mod = yield_mod.set_index("Date")

yc_returns = pd.concat([facts,yield_mod],axis=1).dropna()

In [5]:
model = 'Label'

opt_return = get_fact(model,yc_returns,"Return")
opt_sharpe = get_fact(model,yc_returns,"Sharpe")
opt_var = get_fact(model,yc_returns,"VaR")

opt_return1,out1 = get_opt_overtime(model,yc_returns,"Return")

hy_opt_1w = 1
hy_opt_2w = 1
hy_opt_3w = 1
hy_opt_4w = 1

hy_opt_w = [hy_opt_1w,hy_opt_2w,hy_opt_3w,hy_opt_4w]


print("Return Strategy for cycle 1-4: {}".format(opt_return))
print("Sharpe Strategy for cycle 1-4: {}".format(opt_sharpe))
print("Var Strategy for cycle 1-4: {}".format(opt_var))

print("Market")
stats(returns_data(model, out1,opt_return,hy_opt_w),12,True)
print("\nBest Return Strategy")
stats(returns_data(model, out1,opt_return,hy_opt_w),12)
print("\nBest Return Sharpe")
stats(returns_data(model, out1,opt_sharpe,hy_opt_w),12)
print("\nBest Return Historical VaR")
stats(returns_data(model, out1,opt_var,hy_opt_w),12)

Return Strategy for cycle 1-4: ['RMW', 'Mom', 'Mom', 'Mom']
Sharpe Strategy for cycle 1-4: ['RMW', 'Mom', 'RMW', 'Mom']
Var Strategy for cycle 1-4: ['CMA', 'CMA', 'CMA', 'CMA']
Market
Market Return: 26.85%
Market Vol: 28.0%
Market Sharpe: 0.96


Best Return Strategy
Strategy Return: 15.51%
Strategy Vol: 15.2%
Strategy Sharpe: 1.02

Best Return Sharpe
Strategy Return: 12.11%
Strategy Vol: 12.84%
Strategy Sharpe: 0.94

Best Return Historical VaR
Strategy Return: 3.31%
Strategy Vol: 12.77%
Strategy Sharpe: 0.26


In [6]:


beta_info = returns_data(model, out1,opt_sharpe,hy_opt_w)
beta_info1 = returns_data(model, yc_returns,opt_sharpe,hy_opt_w)

#rf = facts['RF']

X = beta_info['Mkt-RF']
Y = beta_info['Rotation_return']
X = sm.add_constant(X)

reg = sm.OLS(Y, X).fit()

X1 = beta_info1['Mkt-RF']
Y1 = beta_info1['Rotation_return']
X1 = sm.add_constant(X1)

reg1 = sm.OLS(Y1, X1).fit()

print("test on second half")
print(reg.params)
print("")
print("test on entire data")
print(reg1.params)

test on second half
const     0.013666
Mkt-RF   -0.146671
dtype: float64

test on entire data
const     0.022389
Mkt-RF   -0.145579
dtype: float64


In [9]:
returns_data(model, yc_returns,opt_sharpe,hy_opt_w).to_excel("yc_model_returns1.xlsx")